In [1]:
####################  NQPS  ########################


# ver4.8

############################################
##### decide which real machine to use #####
############################################
backend_name = 'ibm_kawasaki'

############################
##### import libraries #####
############################
%matplotlib inline
%load_ext autoreload
%autoreload 2


from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, compiler, circuit, compiler
from qiskit.converters import circuit_to_dag, dag_to_circuit
from pytket.extensions.qiskit import IBMQBackend, AerBackend
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import math
import time
import sys
sys.path.append('../../../aqcel_dev/aqcel_dev')
import aqcel_dev as icepp
sys.path.append('../')
sys.path.append('../qiskit_legacy')
sys.path.append('../qiskit_code')

import QuantumPartonShower as qps
import QuantumPartonShower_ReM as qpsc
import QuantumPartonShower_ReM_2step_hardcode as qps_rhs
import plotting as pl
from PaperPlots import MakeObservables as mo

sim_b = AerBackend()



displaycolor = {
    'u1': ('#FA74A6', '#000000'),
    'u2': ('#FA74A6', '#000000'),
    'u3': ('#FA74A6', '#000000'),
    'id': ('#191970', '#ffffff'),
    'u': ('#BB8BFF', '#000000'),
    'p': ('#BB8BFF', '#000000'),
    'x': ('#05BAB6', '#000000'),
    'y': ('#05BAB6', '#000000'),
    'z': ('#05BAB6', '#000000'),
    'h': ('#6FA4FF', '#000000'),
    'cx': ('#ff00ff', '#000000'),
    'cry': ('#ff00ff', '#000000'),
    'ch': ('#ff00ff', '#000000'),
    'ccx': ('#ff00ff', '#000000'),
    'mcx': ('#ff00ff', '#000000'),
    'mcx_gray': ('#ff00ff', '#000000'),
    'cy': ('#ff00ff', '#000000'),
    'cz': ('#ff00ff', '#000000'),
    'swap': ('#6FA4FF', '#000000'),
    'cswap': ('#ff00ff', '#000000'),
    'ccswap': ('#ff00ff', '#000000'),
    'dcx': ('#ff00ff', '#000000'),
    'cdcx': ('#ff00ff', '#000000'),
    'ccdcx': ('#ff00ff', '#000000'),
    'iswap': ('#6FA4FF', '#000000'),
    's': ('#6FA4FF', '#000000'),
    'sdg': ('#6FA4FF', '#000000'),
    't': ('#BB8BFF', '#000000'),
    'tdg': ('#BB8BFF', '#000000'),
    'sx': ('#6FA4FF', '#000000'),
    'sxdg': ('#6FA4FF', '#000000'),
    'r': ('#BB8BFF', '#000000'),
    'rx': ('#BB8BFF', '#000000'),
    'ry': ('#BB8BFF', '#000000'),
    'rz': ('#BB8BFF', '#000000'),
    'rxx': ('#BB8BFF', '#000000'),
    'ryy': ('#BB8BFF', '#000000'),
    'rzx': ('#BB8BFF', '#000000'),
    'reset': ('#000000', '#FFFFFF'),
    'target': ('#FFFFFF', '#FFFFFF'),
    'measure': ('#000000', '#FFFFFF'),
    'ccx_to_x': ('#006400', '#ffffff'),      # 緑
    'ccx_to_cx': ('#800000', '#ffffff'),     # 茶
    'ccx_to_removed': ('#006400', '#ffffff'),# 緑
    'cx_to_x': ('#191970', '#ffffff'),       # 青
    'cx_to_removed': ('#191970', '#ffffff'), # 青
}

style = {'displaycolor' : displaycolor, 'showindex' : True}

Number of emissions: 1
log(theta_max): -5.180816459236603
Number of emissions: 1
log(theta_max): -1.7269388197455342
Number of emissions: 2
log(theta_max): -1.7269388197455342
Number of emissions: 1
log(theta_max): -3.4538776394910684


In [2]:
N = 1
eps= 0.001
g_1= 2
g_2= 1
initialParticles= ['100'] # f1

shots= 10000

qps0= qps.QuantumPartonShower(N, 1) # for g12= 0
qps1= qps.QuantumPartonShower(N, 1) # for g12= 1

qc0, qubits0= qps0.createCircuit(eps, g_1, g_2, 0, initialParticles)
qc1, qubits1= qps1.createCircuit(eps, g_1, g_2, 1, initialParticles)

t1= time.time()
counts0= qps0.simulate('qasm', shots=shots)
t2= time.time()
counts1= qps1.simulate('qasm', shots=shots)
t3= time.time()

generated circuit on 11 qubits
generated circuit on 11 qubits


In [3]:
#######################
##### apply aqcel #####
#######################

# AQCELにかかる時間を測定開始
start_time = time.time()

# AQCELのapply
target_qc = qc0
aqcel_results_cc= icepp.pass_manager(target_qc, level=2, backend_tket=sim_b, shots=1024, measure_type='cc').auto_manager()
result_qc = aqcel_results_cc[0]

qc1, qc2, qc3, qc4, qc6 = aqcel_results_cc[9], aqcel_results_cc[10], aqcel_results_cc[11], aqcel_results_cc[12], aqcel_results_cc[1]

# AQCEL適用にかかった時間をprint
end_time = time.time()
execution_time = end_time - start_time
print(f"実行時間: {execution_time}秒")

depth2 cry : bitstrings ['1'] and len(bitstrings) is 1
depth3 ccx : bitstrings ['10'] and len(bitstrings) is 1
depth4 ccx : bitstrings ['00'] and len(bitstrings) is 1
depth1 cry : bitstrings ['0'] and len(bitstrings) is 1
depth5 cx : bitstrings ['0'] and len(bitstrings) is 1
depth6 ccx : bitstrings ['00'] and len(bitstrings) is 1
depth7 ccx : bitstrings ['10'] and len(bitstrings) is 1
depth9 ccx : bitstrings ['11'] and len(bitstrings) is 1
depth10 cx : bitstrings ['1'] and len(bitstrings) is 1
depth11 ccx : bitstrings ['11'] and len(bitstrings) is 1
depth13 ccx : bitstrings ['10'] and len(bitstrings) is 1
depth14 cx : bitstrings ['0'] and len(bitstrings) is 1
depth15 ccx : bitstrings ['10'] and len(bitstrings) is 1
depth16 ccx : bitstrings ['01'] and len(bitstrings) is 1
depth17 ccx : bitstrings ['00'] and len(bitstrings) is 1
depth18 cry : bitstrings ['0'] and len(bitstrings) is 1
depth19 ccx : bitstrings ['00'] and len(bitstrings) is 1
depth20 ccx : bitstrings ['01'] and len(bitstrin

In [4]:
#############################
##### Before AQCEL(qc) #####
#############################
qc = target_qc
print('Before AQCEL(qc)')
print(f' - Number of gate   : {list(target_qc.count_ops().items())}')
print(f' - Number of qubits : {target_qc.num_qubits}')
print(f' - Depth of all     : {target_qc.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')


#############################
##### After decompose_mcu(qc1) #####
#############################
print('qc1 : After decompose_mcu')
print(f' - Number of gate   : {list(qc1.count_ops().items())}')
print(f' - Number of qubits : {qc1.num_qubits}')
print(f' - Depth of all     : {qc1.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_adjacent(qc2) #####
#############################
print('qc2 : After remove_adjacent')
print(f' - Number of gate   : {list(qc2.count_ops().items())}')
print(f' - Number of qubits : {qc2.num_qubits}')
print(f' - Depth of all     : {qc2.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_controlled_operations(qc3) #####
#############################
print('qc3 : After remove_controlled_operations')
print(f' - Number of gate   : {list(qc3.count_ops().items())}')
print(f' - Number of qubits : {qc3.num_qubits}')
print(f' - Depth of all     : {qc3.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_adjacent(qc4) #####
#############################
print('qc4 : After remove_adjacent')
print(f' - Number of gate   : {list(qc4.count_ops().items())}')
print(f' - Number of qubits : {qc4.num_qubits}')
print(f' - Depth of all     : {qc4.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

############################
##### After AQCEL(qc5) #####
############################
qc5 = result_qc
print('After AQCEL(qc5) : After remove_qubits')
print(f' - Number of gate   : {list(result_qc.count_ops().items())}')
print(f' - Number of qubits : {result_qc.num_qubits}')
print(f' - Depth of all     :{result_qc.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

############################
##### After transpiled(qc6) #####
############################
print('qc6 : After transpiled')
print(f' - Number of gate   : {list(qc6.count_ops().items())}')
print(f' - Number of qubits : {qc6.num_qubits}')
print(f' - Depth of all     :{qc6.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

# かかる時間を測定開始
start_time = time.time()

import qiskit.quantum_info as qi
statevector1 = icepp.simulator(target_qc).statevector_simulator(output='statevector')
statevector2 = icepp.simulator(result_qc).statevector_simulator(output='statevector')
dm1 = qi.partial_trace(statevector1,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm1.probabilities_dict()
dm2 = qi.partial_trace(statevector2,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm2.probabilities_dict()

# かかった時間をprint
end_time = time.time()
execution_time = end_time - start_time
print(f"実行時間: {execution_time}秒")
print(f"Fsim = {qi.state_fidelity(dm1,dm2)}")

Before AQCEL(qc)
 - Number of gate   : [('ccx', 149), ('x', 126), ('measure', 20), ('cry', 17), ('cx', 7), ('ch', 1)]
 - Number of qubits : 20
 - Depth of all     : 173


qc1 : After decompose_mcu
 - Number of gate   : [('ccx', 149), ('x', 126), ('measure', 20), ('cry', 17), ('cx', 7), ('ch', 1)]
 - Number of qubits : 20
 - Depth of all     : 173


qc2 : After remove_adjacent
 - Number of gate   : [('ccx', 121), ('x', 48), ('measure', 20), ('cry', 17), ('cx', 7), ('ch', 1)]
 - Number of qubits : 20
 - Depth of all     : 147


qc3 : After remove_controlled_operations
 - Number of gate   : [('ccx_to_removed', 98), ('x', 48), ('ccx_to_x', 20), ('measure', 20), ('cx_to_removed', 18), ('cx_to_x', 5), ('ccx_to_cx', 3), ('cry', 1), ('cx', 1)]
 - Number of qubits : 20
 - Depth of all     : 147


qc4 : After remove_adjacent
 - Number of gate   : [('ccx_to_removed', 98), ('x', 48), ('ccx_to_x', 20), ('measure', 20), ('cx_to_removed', 18), ('cx_to_x', 5), ('ccx_to_cx', 3), ('cry', 1), ('cx', 1)]


In [5]:
Nstep_before_AQCEL = qc0
Nstep_after_AQCEL  = qc5

In [6]:
#########################  OQPS  ################################



# ver5.2

############################################
##### decide which real machine to use #####
############################################
backend_name = 'ibm_kawasaki'

############################
##### import libraries #####
############################
%matplotlib inline
%load_ext autoreload
%autoreload 2


from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, compiler, circuit, compiler
from qiskit.converters import circuit_to_dag, dag_to_circuit
from pytket.extensions.qiskit import IBMQBackend, AerBackend
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import math
import time
import sys
sys.path.append('../../../aqcel_dev/aqcel_dev')
sys.path.append('../../../CD_Transpiler_demo_ising_PSJangM')
import aqcel_dev as icepp

from sample.onestepSim_LBNL import runQuantum1
from sample.twostepSim_LBNL import runQuantum2

sim_b = AerBackend()



displaycolor = {
    'u1': ('#FA74A6', '#000000'),
    'u2': ('#FA74A6', '#000000'),
    'u3': ('#FA74A6', '#000000'),
    'id': ('#191970', '#ffffff'),
    'u': ('#BB8BFF', '#000000'),
    'p': ('#BB8BFF', '#000000'),
    'x': ('#05BAB6', '#000000'),
    'y': ('#05BAB6', '#000000'),
    'z': ('#05BAB6', '#000000'),
    'h': ('#6FA4FF', '#000000'),
    'cx': ('#ff00ff', '#000000'),
    'cry': ('#ff00ff', '#000000'),
    'ch': ('#ff00ff', '#000000'),
    'ccx': ('#ff00ff', '#000000'),
    'rccx': ('#ff00ff', '#000000'),
    'rccy': ('#ff00ff', '#000000'),
    'rccz': ('#ff00ff', '#000000'),
    'mcx': ('#ff00ff', '#000000'),
    'mcx_gray': ('#ff00ff', '#000000'),
    'cy': ('#ff00ff', '#000000'),
    'cz': ('#ff00ff', '#000000'),
    'c4ry': ('#ff00ff', '#000000'),
    'c5ry': ('#ff00ff', '#000000'),
    'swap': ('#6FA4FF', '#000000'),
    'cswap': ('#ff00ff', '#000000'),
    'ccswap': ('#ff00ff', '#000000'),
    'dcx': ('#ff00ff', '#000000'),
    'cdcx': ('#ff00ff', '#000000'),
    'ccdcx': ('#ff00ff', '#000000'),
    'iswap': ('#6FA4FF', '#000000'),
    's': ('#6FA4FF', '#000000'),
    'sdg': ('#6FA4FF', '#000000'),
    't': ('#BB8BFF', '#000000'),
    'tdg': ('#BB8BFF', '#000000'),
    'sx': ('#6FA4FF', '#000000'),
    'sxdg': ('#6FA4FF', '#000000'),
    'r': ('#BB8BFF', '#000000'),
    'rx': ('#BB8BFF', '#000000'),
    'ry': ('#BB8BFF', '#000000'),
    'rz': ('#BB8BFF', '#000000'),
    'rxx': ('#BB8BFF', '#000000'),
    'ryy': ('#BB8BFF', '#000000'),
    'rzx': ('#BB8BFF', '#000000'),
    'reset': ('#000000', '#FFFFFF'),
    'target': ('#FFFFFF', '#FFFFFF'),
    'measure': ('#000000', '#FFFFFF'),
    'ccx_to_x': ('#006400', '#ffffff'),      # 緑
    'ccx_to_cx': ('#800000', '#ffffff'),     # 茶
    'ccx_to_removed': ('#006400', '#ffffff'),# 緑
    'cx_to_x': ('#191970', '#ffffff'),       # 青
    'cx_to_removed': ('#191970', '#ffffff'), # 青
}

style = {'displaycolor' : displaycolor, 'showindex' : True}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
qc0 = runQuantum1(gLR=1,dophisplit=1)#circuit_LBNL1

In [8]:
#######################
##### apply aqcel #####
#######################

# AQCELにかかる時間を測定開始
start_time = time.time()

# AQCELのapply
target_qc = qc0
aqcel_results_cc= icepp.pass_manager(target_qc, level=2, backend_tket=sim_b, shots=1024, measure_type='cc').auto_manager()
result_qc = aqcel_results_cc[0]

qc1, qc2, qc3, qc4, qc6 = aqcel_results_cc[9], aqcel_results_cc[10], aqcel_results_cc[11], aqcel_results_cc[12], aqcel_results_cc[1]

# AQCEL適用にかかった時間をprint
end_time = time.time()
execution_time = end_time - start_time
print(f"実行時間: {execution_time}秒")

depth2 cry : bitstrings ['1'] and len(bitstrings) is 1
depth4 rccx : bitstrings ['10', '11'] and len(bitstrings) is 2
depth5 ccx : bitstrings ['10', '11'] and len(bitstrings) is 2
depth6 cry : bitstrings ['0', '1'] and len(bitstrings) is 2
depth6 rccx : bitstrings ['10', '11'] and len(bitstrings) is 2
depth8 rccx : bitstrings ['11', '10'] and len(bitstrings) is 2
depth9 ccx : bitstrings ['10', '11'] and len(bitstrings) is 2
depth10 cry : bitstrings ['0', '1'] and len(bitstrings) is 2
depth10 rccx : bitstrings ['10', '11'] and len(bitstrings) is 2
depth12 rccx : bitstrings ['10', '11', '00', '01'] and len(bitstrings) is 4
depth13 rccx : bitstrings ['01', '11'] and len(bitstrings) is 2
depth14 rccx : bitstrings ['01', '11'] and len(bitstrings) is 2
depth15 cry : bitstrings ['0', '1'] and len(bitstrings) is 2
depth16 rccx : bitstrings ['01', '11'] and len(bitstrings) is 2
depth17 rccx : bitstrings ['01', '11'] and len(bitstrings) is 2
depth18 rccx : bitstrings ['10', '00', '01', '11'] and

In [9]:
#############################
##### Before AQCEL(qc) #####
#############################
qc = target_qc
print('Before AQCEL(qc)')
print(f' - Number of gate   : {list(target_qc.count_ops().items())}')
print(f' - Number of qubits : {target_qc.num_qubits}')
print(f' - Depth of all     : {target_qc.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')


#############################
##### After decompose_mcu(qc1) #####
#############################
print('qc1 : After decompose_mcu')
print(f' - Number of gate   : {list(qc1.count_ops().items())}')
print(f' - Number of qubits : {qc1.num_qubits}')
print(f' - Depth of all     : {qc1.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_adjacent(qc2) #####
#############################
print('qc2 : After remove_adjacent')
print(f' - Number of gate   : {list(qc2.count_ops().items())}')
print(f' - Number of qubits : {qc2.num_qubits}')
print(f' - Depth of all     : {qc2.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_controlled_operations(qc3) #####
#############################
print('qc3 : After remove_controlled_operations')
print(f' - Number of gate   : {list(qc3.count_ops().items())}')
print(f' - Number of qubits : {qc3.num_qubits}')
print(f' - Depth of all     : {qc3.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_adjacent(qc4) #####
#############################
print('qc4 : After remove_adjacent')
print(f' - Number of gate   : {list(qc4.count_ops().items())}')
print(f' - Number of qubits : {qc4.num_qubits}')
print(f' - Depth of all     : {qc4.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

############################
##### After AQCEL(qc5) #####
############################
qc5 = result_qc
print('After AQCEL(qc5) : After remove_qubits')
print(f' - Number of gate   : {list(result_qc.count_ops().items())}')
print(f' - Number of qubits : {result_qc.num_qubits}')
print(f' - Depth of all     :{result_qc.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

############################
##### After transpiled(qc6) #####
############################
print('qc6 : After transpiled')
print(f' - Number of gate   : {list(qc6.count_ops().items())}')
print(f' - Number of qubits : {qc6.num_qubits}')
print(f' - Depth of all     :{qc6.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

# # かかる時間を測定開始
# start_time = time.time()

# import qiskit.quantum_info as qi
# statevector1 = icepp.simulator(target_qc).statevector_simulator(output='statevector')
# statevector2 = icepp.simulator(result_qc).statevector_simulator(output='statevector')
# dm1 = qi.partial_trace(statevector1,[7, 8, 9, 10])
# dm1.probabilities_dict()
# dm2 = qi.partial_trace(statevector2,[7, 8, 9, 10])
# dm2.probabilities_dict()

# # かかった時間をprint
# end_time = time.time()
# execution_time = end_time - start_time
# print(f"実行時間: {execution_time}秒")
# print(f"Fsim = {qi.state_fidelity(dm1,dm2)}")

Before AQCEL(qc)
 - Number of gate   : [('x', 19), ('measure', 6), ('cry', 5), ('mcx', 4), ('c4ry', 2), ('cx', 1)]
 - Number of qubits : 11
 - Depth of all     : 21


qc1 : After decompose_mcu
 - Number of gate   : [('rccx', 20), ('x', 19), ('cry', 7), ('measure', 6), ('ccx', 4), ('cx', 1)]
 - Number of qubits : 14
 - Depth of all     : 38


qc2 : After remove_adjacent
 - Number of gate   : [('rccx', 20), ('x', 9), ('cry', 7), ('measure', 6), ('ccx', 4), ('cx', 1)]
 - Number of qubits : 14
 - Depth of all     : 36


qc3 : After remove_controlled_operations
 - Number of gate   : [('ccx_to_cx', 20), ('x', 9), ('measure', 6), ('cry', 4), ('rccx', 4), ('cx_to_x', 2), ('cx', 1), ('cx_to_removed', 1)]
 - Number of qubits : 14
 - Depth of all     : 36


qc4 : After remove_adjacent
 - Number of gate   : [('ccx_to_cx', 20), ('x', 9), ('measure', 6), ('cry', 4), ('rccx', 4), ('cx_to_x', 2), ('cx', 1), ('cx_to_removed', 1)]
 - Number of qubits : 14
 - Depth of all     : 36


After AQCEL(qc5) : Af

In [10]:
Optimized_before_AQCEL = qc0
Optimized_after_AQCEL  = qc5

In [11]:
# import qiskit
# import sys
# sys.path.append('../../../aqcel_dev/aqcel_dev')
# import aqcel_dev as icepp
# import qiskit.quantum_info as qi

# # QASM形式のファイルからQuantumCircuitを読み込む
# with open("data/circuit.qasm_NQPSB", "r") as f:
#     qasm_str_NQPSB = f.read()
#     # print(qasm_str_NQPSB)
# with open("data/circuit.qasm_NQPSA", "r") as f:
#     qasm_str_NQPSA = f.read()

# Nstep_before_AQCEL = qiskit.QuantumCircuit.from_qasm_str(qasm_str_NQPSB)
# Nstep_after_AQCEL  = qiskit.QuantumCircuit.from_qasm_str(qasm_str_NQPSA)

# # QASM形式のファイルからQuantumCircuitを読み込む
# with open("data/circuit.qasm_OQPSB", "r") as f:
#     qasm_str_OQPSB = f.r
#     ead()
# with open("data/circuit.qasm_OQPSA", "r") as f:
#     qasm_str_OQPSA = f.read()

# Optimized_before_AQCEL = qiskit.QuantumCircuit.from_qasm_str(qasm_str_OQPSB)
# Optimized_after_AQCEL  = qiskit.QuantumCircuit.from_qasm_str(qasm_str_OQPSA)

In [12]:
# qc1←→qc1'

statevector1 = icepp.simulator(Nstep_before_AQCEL).statevector_simulator(output='statevector')
statevector2 = icepp.simulator(Nstep_after_AQCEL).statevector_simulator(output='statevector')
dm1 = qi.partial_trace(statevector1,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm1.probabilities_dict()
dm2 = qi.partial_trace(statevector2,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm2.probabilities_dict()

print(f"Fsim qc1←→qc1' = {qi.state_fidelity(dm1,dm2)}")

Fsim qc1←→qc1' = 0.11093545425042577


In [13]:
# # qc2←→qc2'

# statevector1 = icepp.simulator(Optimized_before_AQCEL).statevector_simulator(output='statevector')
# statevector2 = icepp.simulator(Optimized_after_AQCEL).statevector_simulator(output='statevector')
# dm1 = qi.partial_trace(statevector1,[6, 7, 8, 9, 10])
# dm1.probabilities_dict()
# dm2 = qi.partial_trace(statevector2,[6, 7, 8, 9, 10])
# dm2.probabilities_dict()

# print(f"Fsim qc2←→qc2' = {qi.state_fidelity(dm1,dm2)}")

In [14]:
# qc1←→qc2

statevector1 = icepp.simulator(Nstep_before_AQCEL).statevector_simulator(output='statevector')
statevector2 = icepp.simulator(Optimized_before_AQCEL).statevector_simulator(output='statevector')
dm1 = qi.partial_trace(statevector1,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm1.probabilities_dict()
dm2 = qi.partial_trace(statevector2,[6, 7, 8, 9, 10])
dm2.probabilities_dict()

print(f"Fsim qc1←→qc2 = {qi.state_fidelity(dm1,dm2)}")

Fsim qc1←→qc2 = 0.7600098642901885


In [15]:
# # qc1'←→qc2'

# statevector1 = icepp.simulator(Nstep_after_AQCEL).statevector_simulator(output='statevector')
# statevector2 = icepp.simulator(Optimized_after_AQCEL).statevector_simulator(output='statevector')
# dm1 = qi.partial_trace(statevector1,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
# dm1.probabilities_dict()
# dm2 = qi.partial_trace(statevector2,[6, 7, 8, 9, 10])
# dm2.probabilities_dict()

# print(f"Fsim qc1'←→qc2' = {qi.state_fidelity(dm1,dm2)}")

In [16]:
# from qiskit import Aer
# from qiskit.quantum_info import state_fidelity, DensityMatrix, partial_trace

# # QuantumCircuitの例
# qc1 = qiskit.QuantumCircuit(5)
# qc1.h(0)
# qc1.cx(0, 1)

# qc2 = qiskit.QuantumCircuit(7)
# qc2.x(0)
# qc2.h(1)

# qc3 = qiskit.QuantumCircuit(8)
# qc3.y(0)
# qc3.y(1)

# qc4 = qiskit.QuantumCircuit(2)
# qc4.z(0)
# qc4.h(1)

# # 与えられたQuantumCircuitから最終状態ベクトルを取得する関数
# def get_final_state(qc):
#     backend = Aer.get_backend('statevector_simulator')
#     job = qiskit.execute(qc, backend)
#     return job.result().get_statevector(qc)

# # 2番目のビットに対する部分フィデリティを計算する関数
# def partial_fidelity(state1, state2, num_qubits1, num_qubits2):
#     # 対象の量子ビット以外をトレースアウトする
#     trace_out_qubits1 = list(set(range(num_qubits1)) - {1})
#     trace_out_qubits2 = list(set(range(num_qubits2)) - {1})
    
#     rho1 = partial_trace(DensityMatrix(state1), trace_out_qubits1)
#     rho2 = partial_trace(DensityMatrix(state2), trace_out_qubits2)
#     return state_fidelity(rho1, rho2)

# # 部分フィデリティのテーブルを作成
# circuits = [Nstep_before_AQCEL, Nstep_after_AQCEL, Optimized_before_AQCEL, Optimized_after_AQCEL]
# names = ['NQPSB', 'NQPSA', 'OQPSB', 'OQPSA']
# num_qubits_list = [5, 7, 8, 2]

# states = [get_final_state(qc) for qc in circuits]

# fidelities = [[partial_fidelity(states[i], states[j], num_qubits_list[i], num_qubits_list[j]) for j in range(4)] for i in range(4)]

# # テーブルを表示
# print("    ", end="")
# for name in names:
#     print(name, end="    ")
# print()

# for i in range(4):
#     print(names[i], end=" ")
#     for j in range(4):
#         print("{:.4f}".format(fidelities[i][j]), end="  ")
#     print()
